# VP Function Verification

Reads `vp_function_mapping.csv` and calls each VP function at the latest block to verify the mapping is correct.

In [ ]:
import pandas as pd
from pathlib import Path
from multicall import Call

from mainnet_launch.data_fetching.get_state_by_block import (
    get_state_by_one_block,
    safe_normalize_with_bool_success,
)
from mainnet_launch.constants import ETH_CHAIN, BASE_CHAIN

mapping = pd.read_csv(Path(__file__).parent / "vp_function_mapping.csv" if "__file__" in dir() else "vp_function_mapping.csv")
mapping

In [ ]:
def get_chain(autopool_name: str):
    if autopool_name == "baseETH":
        return BASE_CHAIN
    return ETH_CHAIN


CALLABLE_SIGNATURES = {
    "get_virtual_price()(uint256)",
    "getRate()(uint256)",
    "stEthPerToken()(uint256)",
    "exchangePrice()(uint256)",
    "getInvariantDivActualSupply()(uint256)",
}

results = []

for _, row in mapping.iterrows():
    vp_function = row["vp_function"]
    pool_address = row["pool_address"]
    dv = row["destination_vault_address"]
    chain = get_chain(row["autopool"])
    latest_block = chain.get_block_near_top()

    if vp_function not in CALLABLE_SIGNATURES:
        results.append({
            "autopool": row["autopool"],
            "destination_name": row["destination_name"],
            "dv": dv,
            "pool": pool_address,
            "vp_function": vp_function,
            "vp_value": "N/A (special)",
            "block": latest_block,
        })
        continue

    call = Call(
        pool_address,
        vp_function,
        [("vp", safe_normalize_with_bool_success)],
    )
    try:
        state = get_state_by_one_block([call], latest_block, chain)
        vp_value = state["vp"]
    except Exception as e:
        vp_value = f"ERROR: {e}"

    results.append({
        "autopool": row["autopool"],
        "destination_name": row["destination_name"],
        "dv": dv,
        "pool": pool_address,
        "vp_function": vp_function,
        "vp_value": vp_value,
        "block": latest_block,
    })

results_df = pd.DataFrame(results)
results_df

In [ ]:
# Aerodrome special case: getK() / totalSupply()
from mainnet_launch.adhoc.fee_and_base_apr_checks.augment_plans import compute_aerodome_vp

aero_dv = "0x945a4f719018edBa445ca67bDa43663C815835Ad"
aero_block = BASE_CHAIN.get_block_near_top()
aero_vp = compute_aerodome_vp(aero_dv, aero_block, BASE_CHAIN)
print(f"Aerodrome weETH/WETH VP (K/totalSupply) at block {aero_block}: {aero_vp:.6e}")

In [ ]:
# Flag suspicious values
numeric_results = results_df[results_df["vp_value"].apply(lambda x: isinstance(x, (int, float)))].copy()
numeric_results["vp_value"] = numeric_results["vp_value"].astype(float)

suspicious = numeric_results[
    (numeric_results["vp_value"] < 0.01) | (numeric_results["vp_value"] > 100)
]

if len(suspicious) > 0:
    print(f"Found {len(suspicious)} suspicious VP values (< 0.01 or > 100):")
    print(suspicious[["autopool", "destination_name", "vp_function", "vp_value"]].to_string(index=False))
else:
    print("All VP values look reasonable.")

print(f"\nSummary: {len(numeric_results)} callable, {len(results_df) - len(numeric_results)} special/error")